# TP3-Réseaux bayésiens

In [11]:
from pomegranate import *

## Prise en main

In [12]:
G = BayesianNetwork('BotanicalProblem')
malade = DiscreteDistribution({'Oui': 0.1, 'Non': 0.9})
sec = DiscreteDistribution({'Oui': 0.1, 'Non': 0.9})
perd_feuilles = ConditionalProbabilityTable(
[
    ['Non', 'Non', 'Non', .98],
    ['Non', 'Non', 'Oui', .02],
    ['Non', 'Oui', 'Non', .15],
    ['Non', 'Oui', 'Oui', .85],
    ['Oui', 'Non', 'Non', .1],
    ['Oui', 'Non', 'Oui', .9],
    ['Oui', 'Oui', 'Non', .05],
    ['Oui', 'Oui', 'Oui', .95]
], [malade, sec])


In [13]:
s = State(malade, name='malade')
d = State(sec, name='sec')
l = State(perd_feuilles, name='perd_feuilles')
G.add_states(s, d, l)
G.add_edge(s, l)
G.add_edge(d, l)
G.bake()


In [14]:
print(G)

BotanicalProblem:{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                "Oui" : 0.1,
                "Non" : 0.9
            }
        ],
        "frozen" : false
    },
    "name" : "malade",
    "weight" : 1.0
}{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                "Oui" : 0.1,
                "Non" : 0.9
            }
        ],
        "frozen" : false
    },
    "name" : "sec",
    "weight" : 1.0
}{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "name" : "ConditionalProbabilityTable",
        "table" : [
            [
                "Oui",
                "Oui",
                "Oui",
                "0.95"
            ],
            [
             

In [15]:
beliefs = G.predict_proba({})
for state, belief in zip(G.states, beliefs):
    print(state.name, belief.parameters if type(belief) != str else belief)


malade [{'Oui': 0.10000000000000034, 'Non': 0.8999999999999996}]
sec [{'Oui': 0.10000000000000032, 'Non': 0.8999999999999997}]
perd_feuilles [{'Oui': 0.18320000000000042, 'Non': 0.8167999999999995}]


In [16]:
beliefs = G.predict_proba({'perd_feuilles': 'Oui'})
for state, belief in zip(G.states, beliefs):
    print(state.name, belief.parameters if type(belief) != str else belief)


malade [{'Oui': 0.4939956331877731, 'Non': 0.5060043668122269}]
sec [{'Oui': 0.4694323144104805, 'Non': 0.5305676855895196}]
perd_feuilles Oui


In [17]:
beliefs = G.predict_proba({ 'perd_feuilles': 'Oui', 'sec': 'Non' })
for state, belief in zip(G.states, beliefs):
    print(state.name, belief.parameters if type(belief) != str else belief)


malade [{'Oui': 0.8333333333333325, 'Non': 0.16666666666666755}]
sec Non
perd_feuilles Oui


## Licencié ou augmenté ?
En utilisant Pomegranate, écrivez un programme implémentant ce réseau.

In [18]:
G = BayesianNetwork('Bayes&Co')
licenciement = DiscreteDistribution({'Oui': .5, 'Non': .5})
rh = ConditionalProbabilityTable(
    [
        ['Oui', 'Janet', .3],
        ['Non', 'Janet', .2],
        ['Oui', 'Jim', .35],
        ['Non', 'Jim', .2],
        ['Oui', 'Julia', .35],
        ['Non', 'Julia', .6],
    ],
    [licenciement]
)

comptable = ConditionalProbabilityTable(
    [
        ['Oui', 'James', .37],
        ['Non', 'James', .42],
        ['Oui', 'Jules', .63],
        ['Non', 'Jules', .58],
    ],
    [licenciement]
)

avertissement = ConditionalProbabilityTable(
    [
        ['Janet', 'James', 'lettre', .4],
        ['Janet', 'James', 'mail', .6],
        ['Janet', 'Jules', 'lettre', .7],
        ['Janet', 'Jules', 'mail', .3],
        ['Jim', 'James', 'lettre', .3],
        ['Jim', 'James', 'mail', .7],
        ['Jim', 'Jules', 'lettre', .65],
        ['Jim', 'Jules', 'mail', .35],
        ['Julia', 'James', 'lettre', .25],
        ['Julia', 'James', 'mail', .75],
        ['Julia', 'Jules', 'lettre', .55],
        ['Julia', 'Jules', 'mail', .45],
    ],
    [rh, comptable]
)

In [19]:
state_licenciement = State(licenciement, name='licenciement')
state_rh = State(rh, name='rh')
state_comptable = State(comptable, name='comptable')
state_avertissement = State(avertissement, name='avertissement')

G.add_states(state_licenciement, state_rh, state_comptable, state_avertissement)
G.add_edge(state_licenciement, state_comptable)
G.add_edge(state_licenciement, state_rh)
G.add_edge(state_rh, state_avertissement)
G.add_edge(state_comptable, state_avertissement)
G.bake()

Jeremy a reçu ce matin une lettre de Janet et Jules;
Quelle est la probabilité qu’il s’agisse d’une lettre de licenciement ?

In [20]:
beliefs = G.predict_proba({
    'avertissement': 'lettre',
    'rh': 'Janet',
    'comptable': 'Jules'
})
for state, belief in zip(G.states, beliefs):
    print(state.name, belief.parameters if type(belief) != str else belief)

licenciement [{'Oui': 0.619672131147541, 'Non': 0.380327868852459}]
rh Janet
comptable Jules
avertissement lettre


Et s'il a reçu une lettre de Jim et James ?

In [21]:
beliefs = G.predict_proba({
    'avertissement': 'lettre',
    'rh':'Jim',
    'comptabilite':'James'
})
for state, belief in zip(G.states, beliefs):
    print(state.name, belief.parameters if type(belief) != str else belief)


licenciement [{'Oui': 0.6442401202369373, 'Non': 0.35575987976306267}]
rh Jim
comptable [{'Jules': 0.7734948280434973, 'James': 0.22650517195650266}]
avertissement lettre


Si tout ce que Jeremy sait, c'est que Julia est en charge d'un dossier le concernant, quels sont les risques d'être licencié ?

In [22]:
beliefs = G.predict_proba({
    'rh':'Julia'
})
for state, belief in zip(G.states, beliefs):
    print(state.name, belief.parameters if type(belief) != str else belief)

licenciement [{'Oui': 0.3684210526315789, 'Non': 0.6315789473684211}]
rh Julia
comptable [{'Jules': 0.5984210526315789, 'James': 0.4015789473684212}]
avertissement [{'lettre': 0.42952631578947353, 'mail': 0.5704736842105264}]


Jeremy a reçu un mail ce matin, mais il ne sait pas qui a traité son dossier. Quelle probabilité a-t-il d'y lire une annonce d'augmentation de salaire ?

In [23]:
beliefs = G.predict_proba({
    'avertissement': 'mail'
})
for state, belief in zip(G.states, beliefs):
    print(state.name, belief.parameters if type(belief) != str else belief)


licenciement [{'Oui': 0.4790334104431272, 'Non': 0.5209665895568727}]
rh [{'Jim': 0.2626267431628098, 'Janet': 0.20487442960713706, 'Julia': 0.5324988272300533}]
comptable [{'Jules': 0.45707308551228315, 'James': 0.5429269144877168}]
avertissement mail


## Enquête
En utilisant Pomegranate, écrivez un programme implémentant ce réseau.

In [24]:
G = BayesianNetwork('BN Tech')
# 1% de chance que l'employé soit coupable | 99% soit innocent
cambriolage = DiscreteDistribution({'Cambrioleur': 0.01, 'Innocent': 0.99})

# 45% que le coupable porte un gant et 55% n'en porte pas
gant = ConditionalProbabilityTable([
    ['Cambrioleur', 'Porte Gant',         .45],
    ['Cambrioleur', 'Ne porte pas Gant',  .55],
    ['Innocent',    'Porte Gant',         .999],
    ['Innocent',    'Ne porte pas Gant',  .001],
], [cambriolage])

adn = ConditionalProbabilityTable([
    ['Cambrioleur', 'Oui', 0.999],
    ['Cambrioleur', 'Non', 0.001],
    ['Innocent',    'Oui', 0.001],
    ['Innocent',    'Non', 0.999],
], [cambriolage])

empreinte = ConditionalProbabilityTable([
    ['Cambrioleur', 'Porte Gant',         'Trouve empreinte',     0.02],
    ['Cambrioleur', 'Porte Gant',         'Trouve pas empreinte', 0.98],
    ['Cambrioleur', 'Ne porte pas Gant',  'Trouve pas empreinte', 0.001],
    ['Cambrioleur', 'Ne porte pas Gant',  'Trouve empreinte',     0.999],
    ['Innocent',    'Porte Gant',         'Trouve empreinte',     0.99],
    ['Innocent',    'Porte Gant',         'Trouve pas empreinte', 0.01],
    ['Innocent',    'Ne porte pas Gant',  'Trouve pas empreinte', 0.05],
    ['Innocent',    'Ne porte pas Gant',  'Trouve empreinte',     0.95],
], [cambriolage, gant])

In [25]:
state_cambriolage = State(cambriolage, name='cambriolage')
state_gant = State(gant, name='gant')
state_empreinte = State(empreinte, name='empreinte')
state_adn = State(adn, name='adn')

G.add_states(state_cambriolage, state_gant, state_empreinte, state_adn)
G.add_edge(state_cambriolage, state_gant)
G.add_edge(state_gant, state_empreinte)
G.add_edge(state_cambriolage, state_empreinte)
G.add_edge(state_cambriolage, state_adn)
G.bake()

Probabilité trouver une empreinte

In [26]:
beliefs = G.predict_proba({
    'empreinte': 'Trouve pas empreinte'
})
for state, belief in zip(G.states, beliefs):
    print(state.name, belief.parameters if type(belief) != str else belief)

cambriolage [{'Cambrioleur': 0.4225294988736796, 'Innocent': 0.5774705011263203}]
gant [{'Porte Gant': 0.699774630364166, 'Ne porte pas Gant': 0.30022536963583396}]
empreinte Trouve pas empreinte
adn [{'Oui': 0.4226844398759117, 'Non': 0.5773155601240881}]


Probabilité de ne pas trouver une empreinte & valider le résultat de l'ADN

In [27]:
beliefs = G.predict_proba({
    'empreinte': 'Trouve pas empreinte',
    'adn': 'Oui'
})
for state, belief in zip(G.states, beliefs):
    print(state.name, belief.parameters if type(belief) != str else belief)

cambriolage [{'Cambrioleur': 0.9847328556303572, 'Innocent': 0.01526714436964278}]
gant [{'Porte Gant': 0.9857010999343379, 'Ne porte pas Gant': 0.014298900065662125}]
empreinte Trouve pas empreinte
adn Oui


Probabilté de ne pas trouver une empreinte, valider l'ADN & estimer qu'il ne porter pas de gant.

In [28]:
beliefs = G.predict_proba({
    'empreinte': 'Trouve pas empreinte',
    'adn': 'Oui',
    "gant": "Ne porte pas Gant"
})
for state, belief in zip(G.states, beliefs):
    print(state.name, belief.parameters if type(belief) != str else belief)

cambriolage [{'Cambrioleur': 0.991071428571426, 'Innocent': 0.008928571428573983}]
gant Ne porte pas Gant
empreinte Trouve pas empreinte
adn Oui
